In [2]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [31]:
#Mcdonalds Big Mac Index
Mcdf = pd.read_json("big-mac-index-by-state-2024.json")  # Read JSON file directly
Mcdf= Mcdf.rename(columns={"BigMacIndexByState2023": "McdonaldsBigMac2023"})

In [42]:
#Minimum Wage data
#excel to dataframe
minwagedf = pd.read_csv('minimum-wage-2024-by-state3.csv')
minwagedf = minwagedf.sort_values(by='state').reset_index(drop=True)


In [28]:
#median/ average income
states=[]
averages= []
medians= []

url = "https://www.sofi.com/learn/content/average-salary-in-us/"
r = requests.get(url)
text = r.text
soup= BeautifulSoup(r.text)
trs = soup.find_all('tr')

for row in trs:  
    cols = row.find_all("td")  # Get all columns in the row
    if len(cols) >= 3:  # Ensure it has enough columns
        states.append(cols[0].text.strip())
        averages.append(int(cols[1].text.replace("$", "").replace(",", "")))
        medians.append(int(cols[2].text.replace("$", "").replace(",", "")))
medavgdf = pd.DataFrame({'State':states, 'Average':averages, 'Median':medians})
medavgdf= medavgdf.rename(columns={"State": "state"})


In [55]:
#cost of living
livingcostdf = pd.read_json("cost-of-living-index-by-state-2025.json")  # Read JSON file directly
livingcostdf = livingcostdf.drop(columns=['stateFlagCode'])
livingcostdf = livingcostdf.rename(columns={
    "CostOfLivingIndex_CostOfLivingIndex_num_2024": "CostOfLiving",
    "CostOfLivingIndex_GroceriesCostsIndex_num_2024": "GroceriesCosts",
    "CostOfLivingIndex_HousingCostsIndex_num_2024": "HousingCosts",
    "CostOfLivingIndex_TransportationCostsIndex_num_2024": "TransportationCosts",
    "CostOfLivingIndex_UtilitiesCostsIndex_num_2024": "UtilitiesCosts",
    "CostOfLivingIndex_HealthCostsIndex_num_2024": "HealthCareCosts",
    "CostOfLivingIndex_MiscCostsIndex_num_2024": "MiscellaneousGoodsServicesCosts"
})
livingcostdf= livingcostdf.sort_values(by='state').reset_index(drop=True)


In [30]:
#fast food prices
fastfooddf = pd.read_json("fast-food-prices-by-state-2025.json")
fastfooddf = fastfooddf.drop(columns=['stateFlagCode'])
fastfooddf = fastfooddf.rename(columns={
    "FastFoodPricesDominosMedCheese": "DominosMedCheese",
    "FastFoodPricesMcDonaldsHappyMeal": "McDonaldsHappyMeal",
    "FastFoodPricesMcDonaldsBigMac": "McDonaldsBigMac2025",
    "FastFoodPricesChickfilAChickenSandwich": "ChickfilAChickenSandwich",
    "FastFoodPricesTacoBellComboMeal": "TacoBellComboMeal"
})
fastfooddf= fastfooddf.sort_values(by='state').reset_index(drop=True)

In [56]:
#combine all df into one on state
combined_df = medavgdf.merge(livingcostdf, on='state', how='outer') \
                      .merge(minwagedf, on='state', how='outer') \
                      .merge(Mcdf, on='state', how='outer') \
                      .merge(fastfooddf, on='state', how='outer')

combined_df = combined_df[combined_df['state'] != "District of Columbia"].rename(columns={"Average": "AverageIncome", "Median": "MedianIncome"})
combined_df = combined_df.sort_values(by='state').reset_index(drop=True)
combined_df.to_csv('fast_food_analysis.csv', index=False)
combined_df

,state,AverageIncome,MedianIncome,CostOfLiving,GroceriesCosts,HousingCosts,UtilitiesCosts,TransportationCosts,HealthCareCosts,MiscellaneousGoodsServicesCosts,MinimumWage,McdonaldsBigMac2023,DominosMedCheese,McDonaldsHappyMeal,McDonaldsBigMac2025,ChickfilAChickenSandwich,TacoBellComboMeal
0,Alabama,53394.0,59605.0,88.0,97.2,69.5,100.0,90.8,87.0,95.6,NaN,3.99,12.99,2.79,3.99,3.05,5.79
1,Alaska,66130.0,82512.0,123.8,127.1,118.0,152.8,114.3,150.0,118.9,11.73,4.87,15.99,3.40,4.87,3.72,7.06
2,Arizona,63045.0,66340.0,111.5,101.1,131.9,105.3,104.4,92.7,105.2,14.35,4.43,14.99,3.10,4.43,3.39,6.43
3,Arkansas,51251.0,54658.0,88.7,95.2,74.8,90.7,90.0,85.7,96.7,11.00,3.95,12.99,2.76,3.95,3.02,5.73
4,California,76960.0,74819.0,144.8,111.7,208.7,139.8,136.7,107.7,115.3,16.00,5.11,14.99,3.57,5.11,3.90,7.41
5,Colorado,71968.0,77331.0,102.0,100.7,107.7,89.3,94.8,104.3,102.6,14.42,4.59,14.99,3.21,4.59,3.51,6.66
6,Connecticut,73736.0,81285.0,112.3,102.6,117.2,135.9,104.8,109.9,109.0,15.69,4.95,14.49,3.46,4.95,3.78,7.18
7,Delaware,65998.0,75674.0,100.8,102.5,97.4,98.2,102.2,102.4,102.9,13.25,4.63,13.99,3.24,4.63,3.54,6.72
8,Florida,60216.0,62973.0,102.8,105.2,108.1,101.9,98.8,99.0,99.0,12.00,4.47,13.99,3.12,4.47,3.42,6.48
9,Georgia,61256.0,62468.0,91.3,97.3,79.2,97.4,96.0,100.4,94.6,5.15,4.15,13.99,2.90,4.15,3.17,6.02
